In [ ]:
import csv
import requests
import plotly
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go

init_notebook_mode(connected=True)

baseURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/"

urls = {
    "confirmed": baseURL + "time_series_19-covid-Confirmed.csv",
    "deaths": baseURL + "time_series_19-covid-Deaths.csv",
    "recovered": baseURL + "time_series_19-covid-Recovered.csv"
}

entries = {
    "confirmed": [],
    "deaths": [],
    "recovered": []
}

def plot(title, data):
    data["new"] = [0]
    for i in range(1, len(data["confirmed"])):
       data["new"].append(data["confirmed"][i] - data["confirmed"][i-1])
    
    
    data["active"] = []
    for i in range(0, len(data["confirmed"])):
       data["active"].append(data["confirmed"][i] - data["deaths"][i] - data["recovered"][i])

    count = max(data["confirmed"])
    
    plotData = [ go.Scatter(y=data[k], mode="lines+markers", name=k) for k in data ]
    layout = go.Layout(title=f"{title} ({count} cases)")
    plotly.offline.iplot({"data": plotData, "layout": layout})

def plot_province(name, title=None):
    data = {}
    
    for k in entries:
        data[k] = [entry for entry in entries[k] if entry["Province/State"] == name][0]["cases"]
            
    plot(title or name, data)
                 
def plot_country(name, title=None):
    data = {}
    
    for k in entries:
        country_entries = [entry for entry in entries[k] if entry["Country/Region"] == name]
        cases = [entry["cases"] for entry in country_entries]
        data[k] = [sum(v) for v in list(zip(*cases))]

    plot(title or name, data)

def load(url, entries):
    for row in csv.DictReader(line.decode() for line in requests.get(url).iter_lines()):
        entry = {k: v for k, v in row.items() if k in ["Province/State", "Country/Region"]}
        entry["cases"] = [int(v) for k, v in row.items() if k.endswith("/20") and v]  # /20 is 2020 in date key
        entries.append(entry)

for k in urls:
    load(urls[k], entries[k])

plot_country("Canada")
plot_country("US", "United States")
plot_country("Italy")
plot_country("Iran")
plot_country("China")
plot_country("Korea, South")

plot_province("British Columbia")
plot_province("Alberta")
plot_province("California")